In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_columns', 60) #Display 60 columns for better visualitation.
from pandas.api.types import CategoricalDtype
import warnings
warnings.simplefilter("ignore")

Cargar los datos y realizar un análisis exploratorio y una evaluación de la calidad de los datos necesarios para el resto del caso. Específicamente, evaluar la integridad, validez y actualidad de los datos y proponer estrategias de mitigación de los posibles problemas encontrados.

¿Son los campos “is recid” e “is violent recid” en este conjunto de datos adecuados para evaluar la precisión de las estimaciones de riesgo generadas por el sistema COMPAS? Si no es así, definir y calcular una feature que sí lo sea.

El sistema asigna, de media, evaluaciones de riesgo más altas a los hombres que a las mujeres, ya las personas de raza afroamericana que a las de raza caucásica. Sin embargo, también las tasas de reincidencia son más altas para esos colectivos, aunque no está claro que la asignación de riesgo sea “justa” o no.

Mostrar estas diferencias mediante representaciones gráficas y utilizarlas para analizar si la asignación de evaluaciones es justa o no.

¿Para qué tipo de riesgos, el de delitos generales o el de delitos violentos, tiene el sistema más capacidad predictiva?

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Aprendizaje/Practica5/Ejercicio2/compas-scores.csv')

In [3]:
data

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,num_r_cases,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,is_violent_recid,num_vr_cases,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
1,2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,0,5,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O,NaN,-1,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,2,Low,2014-12-31,Risk of Recidivism,5,Medium,2014-12-31
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,NaN,13009779CF10A,F,NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,1,NaN,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13,NaN,1.0,F,Possession of Cocaine,1,NaN,13011511MM10A,M,0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,NaN,NaN,NaN,13000570CF10A,2013-01-12,NaN,1.0,F,Possession of Cannabis,0,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11752,11753,patrick hamilton,patrick,hamilton,2013-09-22,Male,1968-05-02,47,Greater than 45,Other,0,3,0,0,1,-1.0,2013-09-21 04:06:00,2013-09-22 08:16:13,13013317CF10A,2013-09-21,NaN,1.0,F,Burglary Structure Unoccup,0,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,1,Low,2013-09-22,Risk of Recidivism,3,Low,2013-09-22
11753,11754,raymond hernandez,raymond,hernandez,2013-05-17,Male,1993-06-24,22,Less than 25,Caucasian,0,7,3,5,3,-1.0,2013-05-16 08:58:35,2013-05-18 07:59:30,13009569MM10A,2013-05-16,NaN,1.0,M,Battery,1,NaN,14086081TC30A,M,NaN,2014-10-11,Driving License Suspended,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,5,Medium,2013-05-17,Risk of Recidivism,7,Medium,2013-05-17
11754,11755,dieuseul pierre-gilles,dieuseul,pierre-gilles,2014-10-08,Male,1981-01-24,35,25 - 45,Other,0,4,0,0,0,NaN,NaN,NaN,14014708MM10A,2014-10-07,NaN,1.0,M,Battery,0,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,3,Low,2014-10-08,Risk of Recidivism,4,Low,2014-10-08
11755,11756,scott lomagistro,scott,lomagistro,2013-12-03,Male,1986-12-04,29,25 - 45,Caucasian,0,3,0,0,2,-196.0,2013-05-21 03:12:58,2013-05-22 09:04:20,12018116MM10A,NaN,2013-05-21,196.0,M,arrest case no charge,0,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,2,Low,2013-12-03,Risk of Recidivism,3,Low,2013-12-03


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11757 entries, 0 to 11756
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       11757 non-null  int64  
 1   name                     11757 non-null  object 
 2   first                    11757 non-null  object 
 3   last                     11757 non-null  object 
 4   compas_screening_date    11757 non-null  object 
 5   sex                      11757 non-null  object 
 6   dob                      11757 non-null  object 
 7   age                      11757 non-null  int64  
 8   age_cat                  11757 non-null  object 
 9   race                     11757 non-null  object 
 10  juv_fel_count            11757 non-null  int64  
 11  decile_score             11757 non-null  int64  
 12  juv_misd_count           11757 non-null  int64  
 13  juv_other_count          11757 non-null  int64  
 14  priors_count          

Eliminamos las dos siguientes columnas las cuales tienen únicamente valores nulos.

In [5]:
data = data.drop(['num_vr_cases', 'num_r_cases'], axis =1)

In [6]:
data.isnull().sum()/data.shape[0]

id                         0.000000
name                       0.000000
first                      0.000000
last                       0.000000
compas_screening_date      0.000000
sex                        0.000000
dob                        0.000000
age                        0.000000
age_cat                    0.000000
race                       0.000000
juv_fel_count              0.000000
decile_score               0.000000
juv_misd_count             0.000000
juv_other_count            0.000000
priors_count               0.000000
days_b_screening_arrest    0.100366
c_jail_in                  0.100366
c_jail_out                 0.100366
c_case_number              0.063111
c_offense_date             0.221145
c_arrest_date              0.841966
c_days_from_compas         0.063111
c_charge_degree            0.000000
c_charge_desc              0.063707
is_recid                   0.000000
r_case_number              0.685039
r_charge_degree            0.000000
r_days_from_arrest         0

Observamos que hay varias columnas con un gran porcentaje de valores nulos.

Seleccionaremos las columnas que vayamos a necesitar, o no seleccionamos las que no nos aportan información extra o necesaria.

In [7]:
data_red=data[['name','sex','age', 'is_recid', 'is_violent_recid','decile_score', 'v_decile_score', 'vr_offense_date', 'r_offense_date', 'race',  'priors_count', 'compas_screening_date']]

In [8]:
data_red

,name,sex,age,is_recid,is_violent_recid,decile_score,v_decile_score,vr_offense_date,r_offense_date,race,priors_count,compas_screening_date
0,miguel hernandez,Male,69,0,0,1,1,NaN,NaN,Other,0,2013-08-14
1,michael ryan,Male,31,-1,0,5,2,NaN,NaN,Caucasian,0,2014-12-31
2,kevon dixon,Male,34,1,1,3,1,2013-07-05,2013-07-05,African-American,0,2013-01-27
3,ed philo,Male,24,1,0,4,3,NaN,2013-06-16,African-American,4,2013-04-14
4,marcu brown,Male,23,0,0,8,6,NaN,NaN,African-American,1,2013-01-13
...,...,...,...,...,...,...,...,...,...,...,...,...
11752,patrick hamilton,Male,47,0,0,3,1,NaN,NaN,Other,1,2013-09-22
11753,raymond hernandez,Male,22,1,0,7,5,NaN,2014-10-11,Caucasian,3,2013-05-17
11754,dieuseul pierre-gilles,Male,35,0,0,4,3,NaN,NaN,Other,0,2014-10-08
11755,scott lomagistro,Male,29,0,0,3,2,NaN,NaN,Caucasian,2,2013-12-03


In [9]:
data_red.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11757 entries, 0 to 11756
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   name                   11757 non-null  object
 1   sex                    11757 non-null  object
 2   age                    11757 non-null  int64 
 3   is_recid               11757 non-null  int64 
 4   is_violent_recid       11757 non-null  int64 
 5   decile_score           11757 non-null  int64 
 6   v_decile_score         11757 non-null  int64 
 7   vr_offense_date        882 non-null    object
 8   r_offense_date         3703 non-null   object
 9   race                   11757 non-null  object
 10  priors_count           11757 non-null  int64 
 11  compas_screening_date  11757 non-null  object
dtypes: int64(6), object(6)
memory usage: 1.1+ MB


In [10]:
data_red[data_red.name.duplicated(keep = False)].sort_values(by = 'name')

,name,sex,age,is_recid,is_violent_recid,decile_score,v_decile_score,vr_offense_date,r_offense_date,race,priors_count,compas_screening_date
11585,alejandro cabrera,Male,26,-1,0,4,3,NaN,NaN,Hispanic,0,2013-10-07
6536,alejandro cabrera,Male,26,1,1,4,3,2013-10-06,2013-10-06,Caucasian,4,2013-09-23
2794,alexander williams,Male,60,0,0,7,1,NaN,NaN,African-American,15,2014-12-08
10321,alexander williams,Male,28,-1,0,3,3,NaN,NaN,African-American,0,2014-12-20
3746,angel santiago,Male,37,0,0,6,4,NaN,NaN,Caucasian,1,2013-05-26
...,...,...,...,...,...,...,...,...,...,...,...,...
8368,william taylor,Male,47,0,0,2,3,NaN,NaN,Caucasian,5,2013-04-02
1122,willie johnson,Male,50,0,0,9,6,NaN,NaN,African-American,11,2013-09-09
7396,willie johnson,Male,22,0,0,10,9,NaN,NaN,African-American,9,2014-06-11
3428,xavier johnson,Male,21,1,0,6,9,NaN,2013-05-24,African-American,0,2013-01-19
